# 11 DataFrame

## 11.1 DataFrame的生成

In [ ]:
using DataFrames
da0 = DataFrame(
    name=["张三", "李四", "王五", "赵六"], 
    age=[33, 42, missing, 51],
    sex=["M", "F", "M", "M"])
da1 = copy(da0)

In [ ]:
mat0 = [[1, 3, 5, 7] [2, 3, 3, 6]]
mat1 = DataFrame(mat0,:auto)
rename!(mat1, ["colma", "colmb"]) # 重命名

## 11.2 访问DataFrame信息

In [ ]:
using DataFrames
println((nrow(da1), ncol(da1)))
# 用names(df)返回数据框的变量名字符串数组， 而propertynames(df)返回变量名的符号数组
println(@show names(da1))
println(@show propertynames(da1))

In [ ]:
# 获取每列的类型
zip(names(da1), string.(eltype.(eachcol(da1)))) |> 
    DataFrame |>
    d -> rename!(d, ["Variable", "Type"])

## 11.3 访问DataFrame内容

### 11.3.1 访问单个元素

In [ ]:
using DataFrames
da1[2,1]
da1[2,:name]
da1[2,"name"]

In [ ]:
# 修改元素的值
da1[2,:name] = "孙七"
da1

### 11.3.2 访问一列

In [ ]:
# 访问一列：df[!,2], df[!, "age"]或df[!, :age]
da1[!, :age]
da1[2,:]

In [ ]:
# 在行下标处使用叹号不制作列向量的副本， 效率较高， 可以修改提取的列的值。 可以用冒号:作为行下标， 这会生成一列的副本。
# 使用冒号格式，数据不会被修改
x = da1[:, :age]
x[:] = x .+ 100
println(da1)
# 使用叹号格式，则会被修改
x = da1[!, :age]
x[:] = x .+ 100
println(da1)

当列下标为单个整数、符号、字符串、字符串变量时， 行下标写!或者写:都可以取出一列作为数组。

* 叹号格式为“视图”，不制作副本， 修改取出数组同时也会修改原始数据框。
* 冒号格式会制作一列的副本， 除非将冒号格式直接放在赋值的左侧， 都不会修改原始数据框。
* 为安全起见，应使用冒号格式； 对于很大的数据框， 复制会造成较大开销， 可以谨慎地使用叹号格式。

### 11.3.3 用select选列子集

In [ ]:
# select可以挑选列子集，返回副本
println(select(da1, :name, :age))
# 用Not()说明要去掉的列
println(select(da1, Not([:name, :age])))
# 如下的做法可以将某一列提前到第一列：
println(select(da1, :age, :))

### 11.3.4 访问行子集

In [ ]:
# 可以用first(df,k)取出df前面k行组成的子集， 用last(df,k)取出df最后k行组成的子集。
println(first(da1, 2))
println(last(da1, 2))
# 除了用行序号取行子集， 还可以用条件取行子集
print(da0[da0.sex .== "F", :])
# 可以用in.()判断某列的值是否属于某个子集， 此子集使用Ref()函数指定
print(da0[in.(da0.name, Ref(["张三", "李四"])), :])

In [ ]:
# subset()函数输入一个数据框和若干个变量名和关于该变量选择行子集的示性函数（返回逻辑值的函数）， 返回满足条件的子集副本
print(subset(da0, :sex => x -> x .== "M"))
# 如果某一列有缺失值， 则对此列的逻辑判断结果也会有缺失值。 这时可以用coallese()函数指定缺失时的结果替换值，如：
print(subset(da0, :age => x -> coalesce.(x .< 45, false), :sex => x -> x .== "M"))
print(subset(da0, :age => x -> x .< 45, :sex => x -> x .== "M",skipmissing=true))

In [ ]:
# 可以用filter(f, df)取出df的满足条件的行子集， 其中f是一个以一行作为有名元组类型自变量的示性函数
print(filter(row -> row.sex == "F", da1))
# filter!则会修改输入的数据框， 仅保留满足条件的行

### 11.3.5 访问行列子集

In [ ]:
# 对行、列下标可以指定范围， 用下标向量或者变量名符号向量
print(da1[2:4, [1,3]])
print(da1[2:4, [:name, :sex]])
# 如果仅取一列， 结果将不再是数据框， 而是普通数组
print(da1[2:4, :sex])
# 但是， 如果列下标位置使用数组记号， 则可以取出仅有一列的数据框
print(da1[2:4,[:sex]])

In [ ]:
# 视图是数据框的一个子集， 但不制作副本， 修改视图也会修改原始数据框。 
# 对大型数据访问效率更高， 但访问其中一部分时序号进行下标转换， 有一些额外开销。
da2v = @view da1[2:4, [:sex, :age]]

### 11.3.6 添加列

In [ ]:
da1 = copy(da0)
da1[!,:group] = [1,1,2,2]
print(da1)

# 如果需要添加一些统计量列， 可以用transform()函数， 格式是transform(df, 列名 => 变换函数 => 结果变量名)
using Statistics
da1[!,:height] .= [166, 182, 173, 171]
da2 = transform(da1, :height => maximum => :max_height)
print(da2)

# 也可以对某一列的每一行进行变化， 这时用ByRow()说明要进行的变换。 可以对多列分别变换。
da2 = transform(da1, 
    :height => maximum => :max_height,
    :age => ByRow(x -> x + 20) => :newage)
print(da2)

# 可以用Cols()将不同的列选择方式合并使用
# da2 = da1[:, Cols(:name, Between(:sex, :height))]
print(da1[:, Cols(:name, Between(:sex, :height))])

### 11.3.7 添加行

In [ ]:
da1 = copy(da0)
da2 = push!(da1, ("钱多", 59, "M"))
print(da2)

## 11.4 文件读写

### 11.4.1 CSV文件读写

In [ ]:
# 读入
using CSV, DataFrames
d_class = CSV.read("data/class9.csv", DataFrame)

In [ ]:
# 选项：用户可以用types参数指定一个从变量名字符串到类型的字典
d_class = CSV.read("data/class9.csv", DataFrame,
        types=Dict(
            "name" => String, 
            "sex" => String, 
            "height" => Float64,
            "weight" => Float64))

In [ ]:
# 从网络读入
using Downloads
urlf = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
dht = CSV.read(Downloads.download(urlf), DataFrame,
    header=0)
rename!(dht, ["age", "sex", "cp", "trestbps", "chol", 
    "fbs", "restecg", "thalach", "exang", "oldpeak",    
    "slope", "ca", "thal", "num"])
print(first(dht,5))

In [ ]:
# 写出
CSV.write("data/cleveland.csv",dht)

### 11.4.2 Excel文件读写

## 11.5 数据框变量概括

In [ ]:
# 用describe()函数对数据框各个变量进行简单概括， 结果为数据框格式
using CSV, DataFrames,Downloads
urlf = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
dht = CSV.read(Downloads.download(urlf), DataFrame,
    header=0)
rename!(dht, ["age", "sex", "cp", "trestbps", "chol", 
    "fbs", "restecg", "thalach", "exang", "oldpeak",    
    "slope", "ca", "thal", "num"])
describe(dht)

In [ ]:
# 常用统计函数有mean, median, var, std, iqr, minimum, maximum, quantile.summarystats(x)函数对数值型变量x计算各种简单统计量， 以纯文本格式显示
using Statistics, StatsBase
summarystats(dht[!,"age"])

# 两个变量可以用cov(x,y)计算协方差， cor(x,y)计算相关系数
# cov(hcat(eachcol(dcl)...))

In [ ]:
# 可以用combine函数计算统计量。 
# 输入数据框， 以及用变量名 => 统计函数或者变量名 => 统计函数 => 结果变量名方式指定的一个或多个要汇总的内容。
combine(d_class, :height => mean, :weight => mean)

## 11.6 简单修改

In [ ]:
# 用transform!修改列
dc1 = copy(d_class)
transform!(dc1, :height => (x -> x ./ 100),renamecols = false)

In [ ]:
# 用mapcols()对数据框的所有列应用某种变换， 返回变换结果
dc2 = mapcols(x -> x ./100, dc1[:, ["age", "height", "weight"]])
print(first(dc2, 3))

In [ ]:
# 用replace!()函数对数据框指定的列的某些值进行替换
replace!(dc1.sex, "F" => "女", "M" => "男")

# replace!函数也可以输入一个替换函数， 按该替换函数的映射将第二自变量进行值替换
replace!(x -> ismissing(x) ? 0 : x, dc1[!, :age])
print(first(dc1,3))
# 将某些值替换为missing， 需要先用allowmissing!将整个数据框的所有列(或指定列)允许有缺失值， 然后使用.=赋值

In [ ]:
# 对所有列如果要统一替换， 可以选择整个数据框
da2 = DataFrame(x = [1, 2, 0, 4],y = [11, 0, 33, 44])
allowmissing!(da2)
da2 .= ifelse.(da2 .== 0, missing, da2)

## 11.7 排序

In [ ]:
# 用sort!()函数将数据框安装某一列或某几列排序。 这个函数会修改其输入， 输入的数据框会被修改为排序后的值。
dc1 = copy(d_class)
sort!(dc1, :age)
print(first(dc1,5))

In [ ]:
# 可以指定多列，当前一列相同时按后一列排序：
sort!(dc1, [:sex, :age])
print(first(dc1,5))

In [ ]:
# 在指定排序变量时， 可以用order()函数指定选项， 如rev=true指定降序， by=uppercase指定按转换为大写后排序
sort!(dc1, [:sex, order(:age, rev=true)])
print(first(dc1,5))

## 11.8 纵向合并

In [ ]:
# 如果两个数据框df1和df2结构相同， 只是保存了不同的观测， 可以用vcat(df1, df2)返回上下合并的结果
using DataFrames
df1 = DataFrame(id = [1, 2], x = [101, 102])
df2 = DataFrame(id = [3, 4], x = [201, 202])
vcat(df1, df2)

In [ ]:
# 可以用append!(df1, df2)将内容合并到df1中
append!(df1, df2)
df1

## 11.9 横向合并

* innerjoin(a,b, on=...)默认将左右能匹配的观测保留， 不能匹配的删除， 这在数据库术语中称为“内连接”（inner join）。
如果用来连接的关键列名字不一样， 可以提供对作为on的输入，如on = :id => :num。
* leftjoin作左连接，即保留左侧数据框所有观测， 右侧数据框仅保留能匹配的观测。
* rightjoin作右连接，保留右侧数据框所有观测， 左侧数据框仅保留能匹配的观测。
* outerjoin作全外连接。保留两侧数据框所有观测。
* semijoin仅保留左侧数据框能匹配的观测， 不保留右侧数据框内容， 实际是用右侧数据框的键值来选择左侧数据框的内容。
* antijoin仅保留左侧数据框不能匹配的观测， 不保留右侧数据框内容， 也是用右侧数据框的键值来排除左侧数据框的内容。

In [ ]:
using DataFrames
dfj9 = DataFrame(id=["a", "b"], X=[91,92])
dfj10 = DataFrame(id=["a", "c"], Y=[101,102])
println(innerjoin(dfj9,dfj10,on=:id))
println(leftjoin(dfj9, dfj10, on=:id))
println(rightjoin(dfj9, dfj10, on=:id))
println(outerjoin(dfj9, dfj10, on=:id))
# 在左连接、右连接或外连接时， 可以用source = 列名选项使得输出中增加一项用来表示当前观测是否仅来自左侧、仅来自右侧还是来自两侧
println(outerjoin(dfj9, dfj10, on=:id, source = :source))

In [ ]:
# crossjoin则对输入的两个或多个数据框找到观测的所有组合， 输出组合的结果。 不使用on自变量。
dfc1 = DataFrame(a = [1,2,3], b=["a", "a", "b"])
dfc2 = DataFrame(c = [21,22])
println(crossjoin(dfc1, dfc2))

## 11.10 长宽表转换

In [ ]:
# stack函数将同一行的不同列堆叠到同一列中， 实现宽表到长表的转换
using DataFrames
dw1 = DataFrame(id=["a", "b", "c"], x1 = [11, 12, 13], x2=[21, 22, 23])
println(dw1)
dw1t = stack(dw1, [:x1, :x2], :id)
println(dw1t)

In [ ]:
# unstack函数将放在同一列的多个测量值转为存放在同一行， 并适当命名
d1n = DataFrame(id=["a", "a", "b", "b", "c", "c"],time = [1,2,1,2,1,2],value=[11,12, 21,22, 31,32])
println(d1n)
d1nw = unstack(d1n, :id, :time, :value)
println(d1nw)


## 11.11 缺失值管理

In [ ]:
# 用ismissing()判断某个值是否缺失值， 加点以后可以判断某个向量的每个元素
using DataFrames
da0 = DataFrame(
    name=["张三", "李四", "王五", "赵六"], 
    age=[33, 42, missing, 51],
    sex=["M", "F", "M", "M"])
da1 = copy(da0)
ismissing.(da1[!,:age]) |>show

In [ ]:
# 有missing参与的四则运算、比较、数学计算一般返回缺失值。 用skipmissing()可以在计算时删除指定列中的缺失值进行计算
@show sum(da1[!,:age])
@show sum(skipmissing(da1[!,:age]))

In [ ]:
# 用coalesce.()可以将缺失值替换为指定的值
coalesce.(da1[!,:age], 0) |> show

In [ ]:
# 用Missings.replace()输出一个迭代器， 可以在计算中用指定值替换缺失值
sum(Missings.replace(da1[!,:age], 0))

In [ ]:
# 为了获得整个数据框df中每行是否不含任何缺失值的示性值（1表示没有缺失值，0表示有），用DataFrames.completecases(df)函数。
DataFrames.completecases(da1) |> show

In [ ]:
# 将整个数据框中含有缺失值的观测都删去， 用DataFrames.dropmissing!(df)函数。
dropmissing!(da1)
da1 |> show

## 11.12 分类变量

In [ ]:
# 用categorical()函数将分类变量转换成分类数组类型， 允许有缺失值(missing)
using CategoricalArrays,CSV,DataFrames
d_class = CSV.read("data/class9.csv", DataFrame)
dcl = copy(d_class);
dcl[!,:sex] = categorical(dcl[!,:sex]);
show(dcl[!,:sex])
typeof(dcl[!,:sex])

In [ ]:
# 若x为整型变量， StatsBase.counts(x)对x的最小值到最大值的所有整数值计算频数
using StatsBase
dcl = copy(d_class)
dcl[:,:age] |> x -> DataFrame(
    age = minimum(x):maximum(x),
    count = StatsBase.counts(x))

In [ ]:
# 函数StatsBase.frequency(x)计算比例
# 对于更一般的类型如字符串， counts和frequency不支持， 可以使用更一般的StatsBase.countmap(x)函数
StatsBase.countmap(dcl[:,:sex]) |> show

# countmap输出一个字典， 缺点是不能排序。 利用这个函数写一个频数函数
function freqd(x)
    di = StatsBase.countmap(x)
    d = DataFrame(x = collect(keys(di)), count = collect(values(di)))
    sort!(d)
    return d
end
freqd(dcl[:,:sex])

## 11.13 日期和时间类型

### 日期时间

In [ ]:
# Dates.today()返回当天日期。
# 可以从年月日的整数值用Dates.Date()转换成日期， 也可以从日期字符串按照某个模板转换成日期
import Dates
Dates.Date(2018)
## 2018-01-01
Dates.Date(2018, 10)
## 2018-10-01
Dates.Date(2018, 10, 31)
## 2018-10-31
Dates.Date("2018-10-31")
## 2018-10-31
Dates.Date("2018-10-31", "y-m-d")
## 2018-10-31
Dates.Date("20181031", "yyyymmdd")
## 2018-10-31
Dates.Date.([2018, 2018], [3, 10], [15, 31])

In [ ]:
# 可以用DateTime()函数将年、月、日、时、分、秒、毫秒整数值转换成日期时间， 精确到1毫秒
Dates.DateTime(2018, 10, 31, 12, 15, 30, 136)

### 提取成分
* 用Dates.year(d)提取年，Dates.month(d)提取月份数值， Dates.day(d)提取日数值。
* 用Dates.yearmonth(d)提取年、月元组， Dates.monthday(d)提取月、日元组， Dates.yearmonthday(d)提取年、月、日元组。
* 用Dates.dayofweek(d)提取星期几的数值， 星期一返回1，星期日返回7。
* 用Dates.dayname(d)返回星期几的名称，如"Monday"。
* 用Dates.dayofmonth(d)返回当前的星期号码是本月的第几个。

### 日期运算
两个日期或者时间可以比较大小， 可以相减。结果带有单位， 用Dates.value()转换为表示天数或者毫秒数的整数值

In [ ]:
Dates.Date(2018, 10, 31) - Dates.Date(2018) |> Dates.value
## 303
Dates.DateTime(2018, 10, 31, 12, 15, 30, 136) - 
  Dates.DateTime(2018) |> Dates.value 
## 26223330136
Dates.DateTime(2018, 10, 31, 12, 15, 30, 136) - 
Dates.DateTime(2018) |> 
Dates.value |> 
x -> x / (24*3600*1000)

日期和日期时间不能直接加减数值， 而需要用单位表示，类型为Dates.Period。 比如， 加一天应该加Dates.Day(1)； 单位包括：
* Dates.Year()
* Dates.Month()
* Dates.Day()
* Dates.Hour()
* Dates.Minute()
* Dates.Second()
* Dates.Millisecond(): 毫秒

In [ ]:
Dates.Date("2020-01-01", "y-m-d") + Dates.Year(2) + Dates.Month(6) + Dates.Day(15)

### 日期序列
可以用start:step:end格式生成一系列日期， 其中step用日期单位。

In [ ]:
Dates.Date("2020-01-01"):Dates.Day(1):Dates.Date("2020-01-07") |> collect

## 11.14 使用DataFramesMeta包

### 查询、排序、计算新变量

In [ ]:
using DataFrames, DataFramesMeta, CSV
d_class = CSV.read("data/class9.csv", DataFrame)
@chain d_class begin
    @subset(:sex .== "M")
    @transform(:bmi = :weight ./ (:height ./ 100.0).^2)
    @select(:name, :years = :age, :bmi)
    @orderby(:name)
    first(3)
end

* 用@chain指定进行一系列的数据框操作， 用begin和end界定。
* 用@subset取行子集， 注意关于列向量的比较运算需要使用加点的广播形式。 多个条件用逗号分隔。
* 用@transform计算新变量， 注意使用列向量进行计算时要用广播形式。 多个新变量定义用逗号分隔。
* 用@select选择列子集， 并且可以用新变量名=老变量名的格式改名。
* 用@orderby排序。 可以使用多个变量。 如果某个变量x需要降序排列， 用sortperm(:x, rev=true)。

### 用@combine汇总计算

In [ ]:
aa = @chain d_class begin
    @combine(:n = length(:height), 
        :mean = mean(:height),
        :std = std(:height))
end

### 用groupby和@combine汇总计算

In [ ]:
@chain d_class begin
    groupby(:sex)
    @combine(:n = length(:height), 
        :mean = mean(:height),
        :std = std(:height))
end

In [ ]:
@chain d_class begin
    groupby(:sex)
    @transform(:hextra = :height .- minimum(:height))
    @orderby(:sex, :hextra)
end

## 11.15 用Query包进行查询

In [ ]:
using Query, DataFramesMeta, CSV
d_class = CSV.read("data/class9.csv", DataFrame)
df_tmp = @from i in d_class begin
    @where i.sex=="F" && i.age <= 12
    @select {i.name, 体重=i.weight}
    @orderby 体重
    @collect DataFrame
end
@show df_tmp

## 11.16 分组汇总

对数据框经常需要按某一个或几个分类变量进行分类汇总。 可以将数据框分成若干个子数据框， 对每一自数据进行一些汇总处理， 然后将汇总结果合并为一个数据框， 这样的流程称为“分组-操作-合并”(Split-Apply-Combine)。

用groupby()函数分组。 分组后， 可以用combine()进行汇总统计然后合并结果； 可以用select, select!生成与每个子数据框行数相同的仅包含新生成列的结果； 可以用transform, transform!对每个子数据框生成与每个子数据框行数相同的结果数据框， 其中包括原有列与新生成的列。 最后将分组的结果合并。 对每个子数据框汇总或者变换时， 用“cols => func”或格式“cols => func => newcols”格式指定那些了列需要进行什么操作。 实际上， 部分组也可以使用这些函数进行操作， 这可以看成是仅有一个组。

In [60]:
using DataFrames, Statistics
d_class = CSV.read("data/class9.csv", DataFrame)
dcl = copy(d_class)
gdf = groupby(d_class, :sex)
combine(gdf, nrow)

Row,sex,nrow
,String1,Int64
1,F,4
2,M,5


In [61]:
dcl |>
    df -> groupby(df, :sex) |>
    subdf -> combine(subdf, 
    nrow => :观测数, 
    :height => mean => :平均身高, 
    :weight => mean => :平均体重)

Row,sex,观测数,平均身高,平均体重
,String1,Int64,Float64,Float64
1,F,4,142.25,33.5
2,M,5,153.2,43.6
